Grant individual permission to secret in Unity Catalog:

In [0]:
# import Azure SDK library
from azure.keyvault.secrets import SecretClient 

# service credentials from Unity Catalog
credential = dbutils.credentials.getServiceCredentialsProvider('key-vault-access-connector')
# Azure Key Vault url - copy it from vault properties
vault_url = "https://dudekkeyvault.vault.azure.net/"
# init client
client = SecretClient(vault_url=vault_url, credential=credential)

In [0]:
# Take the secret and print the first four chars for test; otherwise, it will be redacted
secret_value = client.get_secret("my-secret").value
print("Got secret:", secret_value[:4], "...[redacted]")

In [0]:
%sql
CREATE SCHEMA secrets;

In [0]:
%sql
CREATE OR REPLACE FUNCTION secrets.get_my_secret(environment STRING)
RETURNS STRING
LANGUAGE PYTHON
HANDLER 'batchhandler'
PARAMETER STYLE PANDAS
ENVIRONMENT (
  dependencies = '[
    "azure-keyvault-secrets==4.8.0"
  ]',
  environment_version = 'None'
)
CREDENTIALS (
  `key-vault-access-connector` AS kv DEFAULT
)
AS $$
def batchhandler(batch_iter):
  for _ in batch_iter:
    import pandas as pd
    from azure.keyvault.secrets import SecretClient
    from databricks.service_credentials import getServiceCredentialsProvider
    vault_url = "https://dudekkeyvault.vault.azure.net/"
    client = SecretClient(vault_url=vault_url, credential=getServiceCredentialsProvider('kv'))
    secret_value = client.get_secret("my-secret").value
    yield pd.Series( [secret_value] )
$$;

In [0]:
%sql
SELECT hub.secrets.get_my_secret('dev');